# Lesson 13: Advanced Agent Features

🔴 **Advanced** · ⏱ **25 min**

---

This lesson covers three advanced features that improve response quality and trustworthiness: **Reflection** (agent reviews its own output), **Bias Check** (agent examines its response for potential biases), and **Groundedness** (agent verifies its response against known facts). These are especially important for production agents in sensitive domains.

## What you'll learn

- Enable **Reflection** so agents self-review and improve their own responses
- Enable **Bias Check** so agents detect and correct potential biases before returning a response
- Add **Groundedness** facts so agents verify their responses against known truths
- Understand when each feature is appropriate and when to skip it
- Combine all three features for maximum quality assurance in high-stakes agents

## Prerequisites

> **Note:** This is an **optional advanced lesson**. It is not required to complete the core lyzr-adk series.

Before starting this lesson, you should have completed (or be familiar with):

- **Lesson 1** — Getting Started (agent creation basics)
- **Lesson 2** — Providers and Models
- **Lesson 3** — Agent Lifecycle
- **Lesson 4** — Structured Outputs

You will also need:
- A valid `LYZR_API_KEY` set as an environment variable (or replace the placeholder in the setup cell)

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## Advanced Features Overview

lyzr-adk provides three opt-in quality assurance features that wrap around your agent's normal response pipeline. Each one adds an extra LLM call that checks or improves the output before it is returned to you.

| Feature | Method | What it does | Use when |
|---------|--------|--------------|----------|
| Reflection | `enable_reflection()` | Agent reviews and improves its own draft before returning | High-stakes responses, complex reasoning |
| Bias Check | `enable_bias_check()` | Agent checks response for potential biases | HR, medical, legal, sensitive domains |
| Groundedness | `add_groundedness_facts([...])` | Agent verifies response against provided facts | When accuracy against known truths is critical |

All three features are disabled by default. You opt in to each one explicitly, which means your agents stay fast unless you specifically need these guarantees.

## Feature 1: Reflection

When reflection is enabled, the agent generates an initial draft response and then critiques it — looking for gaps, inaccuracies, or unclear reasoning — before producing a final, improved response. The process is fully automatic: you call `enable_reflection()` once and the agent handles the self-review loop internally.

**Trade-off:** Reflection results in higher quality responses, but it requires an additional LLM call, so responses take longer. Use it when quality matters more than speed.

In [ ]:
# Agent WITHOUT reflection
basic_agent = studio.create_agent(
    name="Basic Agent",
    provider="openai/gpt-4o",
    role="Technical writer",
    goal="Write clear, accurate technical explanations",
    instructions="Write clearly and accurately."
)

r_basic = basic_agent.run("Explain the difference between synchronous and asynchronous programming.")
print("Without reflection:")
print(r_basic.response[:400])
print()

In [ ]:
# Same agent WITH reflection enabled
reflection_agent = studio.create_agent(
    name="Reflection Agent",
    provider="openai/gpt-4o",
    role="Technical writer",
    goal="Write clear, accurate technical explanations",
    instructions="Write clearly and accurately."
)

reflection_agent.enable_reflection()  # Agent will self-review its response

r_reflection = reflection_agent.run("Explain the difference between synchronous and asynchronous programming.")
print("With reflection:")
print(r_reflection.response[:400])
print()
print("Reflection adds a self-review step — response takes longer but is typically more complete and accurate.")

## Feature 2: Bias Check

When bias check is enabled, the agent automatically examines its response for potential biases — gender, racial, age, cultural, socioeconomic, and others — before returning it. If a bias is detected, the agent revises the response to be more fair and balanced.

This is particularly important for:
- **HR tools** — hiring recommendations, performance reviews, job descriptions
- **Medical advisors** — symptom interpretation, treatment suggestions
- **Legal assistants** — advice that must apply equally regardless of demographics
- **Financial advisors** — investment guidance that should not favor or disadvantage particular groups

Call `enable_bias_check()` once on the agent and it applies to every subsequent `run()` call.

In [ ]:
# Bias check example — useful for HR/hiring/medical use cases
bias_check_agent = studio.create_agent(
    name="Unbiased HR Assistant",
    provider="openai/gpt-4o",
    role="Objective HR advisor",
    goal="Provide fair, unbiased HR guidance",
    instructions="Provide objective, evidence-based HR advice. Be fair and inclusive."
)

bias_check_agent.enable_bias_check()
print("Bias check enabled!")

# Ask something that could have biased angles
r_bias = bias_check_agent.run(
    "What qualities should we look for when hiring for a leadership role?"
)
print(f"\nResponse with bias check:\n{r_bias.response}")

## Feature 3: Groundedness

Groundedness lets you provide a list of factual statements that the agent must stay consistent with. Before returning its response, the agent checks whether anything it said contradicts the provided facts. If a contradiction is found, the agent corrects or flags it.

Use groundedness when:
- Your agent answers questions about a specific product and must not hallucinate incorrect specs
- Your agent provides medical or legal information and must respect a set of established guidelines
- Your agent represents your company and must stay aligned with official policy statements

Pass a list of strings to `add_groundedness_facts([...])`. The more specific and unambiguous the facts, the more effective the grounding.

In [ ]:
# Define the facts your agent must stay grounded in
company_facts = [
    "Lyzr ADK was released in 2026.",
    "Lyzr ADK supports Python 3.8 and above.",
    "The package is installed with: pip install lyzr-adk",
    "Lyzr ADK supports these providers: OpenAI, Anthropic, Google, Groq, Perplexity, AWS Bedrock.",
    "The main class is Studio, imported from the lyzr package.",
    "Lyzr API keys start with the prefix 'sk-'.",
    "Lyzr ADK does NOT support JavaScript or TypeScript — Python only.",
]

# Create a grounded agent
grounded_agent = studio.create_agent(
    name="Grounded Product Expert",
    provider="openai/gpt-4o",
    role="Lyzr ADK product expert",
    goal="Answer questions about Lyzr ADK accurately",
    instructions="Answer based only on verified product facts. Be precise."
)

grounded_agent.add_groundedness_facts(company_facts)
print(f"Groundedness configured with {len(company_facts)} facts")

In [ ]:
# Test groundedness — agent should stay consistent with facts
questions = [
    "What Python version does lyzr-adk require?",
    "How do I install lyzr-adk?",
    "Which LLM providers does lyzr-adk support?",
]

for q in questions:
    r = grounded_agent.run(q)
    print(f"Q: {q}")
    print(f"A: {r.response}\n")

## Combining All Three Features

The three features are fully composable — you can enable any combination of them on the same agent. When all three are active, the agent:

1. Generates an initial response
2. Runs a **reflection** pass to improve quality and completeness
3. Runs a **bias check** to ensure fairness
4. Runs a **groundedness** check against your provided facts
5. Returns the final, verified response

This is the highest level of quality assurance available in lyzr-adk. Use it for agents deployed in regulated or high-stakes domains where errors have real consequences.

In [ ]:
# Medical information agent with all three features
medical_facts = [
    "Always recommend consulting a doctor for medical advice.",
    "This information is for educational purposes only.",
    "Symptoms can vary significantly between individuals.",
    "Self-diagnosis is not a substitute for professional medical evaluation.",
]

# Create an agent with all three advanced features
premium_agent = studio.create_agent(
    name="Medical Info Assistant",
    provider="openai/gpt-4o",
    role="Medical information specialist",
    goal="Provide accurate, unbiased, responsible medical information",
    instructions=(
        "Provide educational medical information. "
        "Always recommend professional consultation. "
        "Be balanced, unbiased, and factually accurate."
    )
)

premium_agent.enable_reflection()
premium_agent.enable_bias_check()
premium_agent.add_groundedness_facts(medical_facts)

print("Agent configured with all three advanced features:")
print("   Reflection: active")
print("   Bias check: active")
print("   Groundedness: 4 facts loaded")

response = premium_agent.run(
    "What are the general symptoms of stress and how might they differ between people?"
)
print(f"\nResponse:\n{response.response}")

## When to Use Each Feature

Not every agent needs all three features. Use this decision guide to choose the right combination:

```
Use Reflection when:    response quality > response speed
                        (complex reasoning, high-stakes content, technical explanations)

Use Bias Check when:    sensitive domain (HR, medical, legal, finance, education)
                        (any agent whose output could disadvantage a group of people)

Use Groundedness when:  agent must stay consistent with known facts
                        (product docs, policy, regulatory requirements, brand guidelines)
```

### Quick decision table

| Use case | Reflection | Bias Check | Groundedness |
|----------|------------|------------|--------------|
| Simple chatbot | No | No | No |
| Technical explainer | Yes | No | No |
| HR hiring assistant | Yes | Yes | No |
| Product Q&A bot | No | No | Yes |
| Medical info agent | Yes | Yes | Yes |
| Legal research tool | Yes | Yes | Yes |

## Common Mistake: Enabling All Features When Only One Is Needed

Each advanced feature adds a full LLM call to the response pipeline. Enabling all three when you only need one will make your agent **3x slower** without any meaningful benefit for simple use cases.

**Rule:** Enable only the features that serve your specific use case. Start with none, and add features only when you have a concrete reason to do so.

In [ ]:
# Overusing features — slows down simple use cases
overkill_agent = studio.create_agent(
    name="Overkill Agent",
    provider="openai/gpt-4o",
    role="Simple chatbot", goal="Answer questions", instructions="Be helpful."
)
overkill_agent.enable_reflection()  # Unnecessary for a simple chatbot
overkill_agent.enable_bias_check()  # Not needed for casual conversation
overkill_agent.add_groundedness_facts(["The sky is blue."])  # Irrelevant facts

# Only use features that serve the use case
simple_agent = studio.create_agent(
    name="Simple Chatbot",
    provider="openai/gpt-4o",
    role="Friendly chatbot", goal="Have nice conversations", instructions="Be friendly and helpful."
)
# No advanced features needed — keep it fast and simple

print("Match features to use case. Each feature adds response latency.")
print("   Reflection:   +1 LLM call")
print("   Bias check:   +1 LLM call")
print("   Groundedness: +1 LLM call")
print("   All three:    +3 LLM calls (3x slower)")

## Exercise: Financial Advice Agent

Put it all together. Your task is to build a financial advice agent that:

1. Uses **groundedness** to enforce regulatory disclaimers (facts are provided below)
2. Uses **bias check** to ensure advice is fair regardless of wealth level, age, or background
3. Optionally adds **reflection** if you want higher quality explanations
4. Is tested with three finance questions

Fill in the `TODO` sections in the cell below.

In [ ]:
# Financial disclaimer facts
financial_disclaimers = [
    "This is not personalized financial advice.",
    "Past performance does not guarantee future results.",
    "All investments carry risk, including loss of principal.",
    "Consult a licensed financial advisor before making investment decisions.",
    "Tax implications vary by jurisdiction and individual circumstances.",
]

# TODO: Create a financial advice agent with appropriate features
finance_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Enable the appropriate features (bias check + groundedness at minimum)
# finance_agent.enable_...()       # enable bias check
# finance_agent.add_...(...)       # add groundedness facts

# TODO: Test with 3 finance questions
for question in [
    "Should I invest in index funds or individual stocks?",
    "How much should I save for retirement?",
    "What's the best way to start investing with $1,000?",
]:
    r = finance_agent.run(question)
    print(f"Q: {question}")
    print(f"A: {r.response[:250]}...\n")

## Summary

### Advanced features at a glance

| Feature | Method | Extra calls | When to use |
|---------|--------|-------------|-------------|
| Reflection | `agent.enable_reflection()` | +1 LLM call | Complex reasoning, high-stakes content |
| Bias Check | `agent.enable_bias_check()` | +1 LLM call | HR, medical, legal, sensitive domains |
| Groundedness | `agent.add_groundedness_facts([...])` | +1 LLM call | Must stay consistent with known facts |

### Latency trade-offs

```
No features:      1x  (baseline)
Reflection only:  ~2x
Bias check only:  ~2x
Groundedness only: ~2x
All three:        ~4x
```

### Key takeaways

- All three features are **disabled by default** — opt in only when you need them
- Each feature adds one extra LLM call; enable only what your use case requires
- **Reflection** improves quality — best for complex, technical, or high-stakes responses
- **Bias Check** improves fairness — required for HR, medical, legal, and financial agents
- **Groundedness** improves accuracy — pass specific, unambiguous facts for best results
- All three features are composable and compatible with memory, sessions, tools, and RAG

## You've Completed the Lyzr ADK Series!

Congratulations — you've finished all 13 lessons of the lyzr-adk course. You started by creating your first agent and ended by configuring production-grade quality controls. Here's the full journey you completed:

### Core Series (Lessons 1–10)

| Lesson | Topic | Key concept |
|--------|-------|-------------|
| [01](./01_getting_started.ipynb) | Getting Started | `Studio`, `create_agent`, `agent.run()` |
| [02](./02_providers_and_models.ipynb) | Providers and Models | OpenAI, Anthropic, Google, Groq, and more |
| [03](./03_agent_lifecycle.ipynb) | Agent Lifecycle | Create, run, update, delete |
| [04](./04_structured_outputs.ipynb) | Structured Outputs | Pydantic schemas, typed responses |
| [05](./05_memory_and_sessions.ipynb) | Memory and Sessions | Stateful multi-turn conversations |
| [06](./06_tools_and_functions.ipynb) | Tools and Functions | Custom tools, function calling |
| [07](./07_knowledge_bases_rag.ipynb) | Knowledge Bases (RAG) | Document grounding, retrieval |
| [08](./08_contexts.ipynb) | Contexts | Shared context across agents |
| [09](./09_rai_guardrails.ipynb) | RAI Guardrails | Responsible AI, content safety |
| [10](./10_capstone_project.ipynb) | Capstone Project | End-to-end agent build |

### Advanced Series (Lessons 11–13)

| Lesson | Topic | Key concept |
|--------|-------|-------------|
| [11](./11_streaming.ipynb) | Streaming | Real-time token output |
| [12](./12_image_and_file_generation.ipynb) | Image and File Generation | `set_image_model()`, file artifacts |
| **13** | **Advanced Features (this lesson)** | Reflection, Bias Check, Groundedness |

---

### What's next?

- **Documentation**: [docs.lyzr.ai](https://docs.lyzr.ai)
- **Community**: [Lyzr Discord](https://discord.gg/lyzr)
- **GitHub**: [github.com/LyzrCore](https://github.com/LyzrCore)
- **Platform**: [studio.lyzr.ai](https://studio.lyzr.ai)

---

*Built with lyzr-adk. Happy building.*